# SQL Setup and Analysis 

  - 1.0 Import CSV Files into Pandas Data Frame
  - 1.1 Create a SQLite Database and Import DataFrames
  - 2.0 Load SQL Extension and Connect to Database
  - 2.1 View Tables 
  - 3.0 Explore Data and Generating Insights
  - 3.1. Top-Selling Products
  - 3.1.1. Sales by Region
  - 3.1.2. Inventory Levels for Reordering
  - 3.1.3. Supplier Performance Metrics
  - 3.1.4. Monthly Sales by Product Category
  - 3.1.5. Rolling Monthly Sales by Product Category
  - 3.1.6. Customer Segment Insights
  - 3.1.7. Average Order Value by Product Category and Payment Type
  - 3.1.8. Payment Method by Client Type
  - 3.1.9. Top 10 Suppliers at Risk for Delays
  - 3.1.10. Top 10 Projected Shortages
  - 3.1.11. Upcoming Production Scheduled at Risk due to Material Shortage
  - 3.1.12. Total Expected Incoming Inventory from Purchase Orders




## 1.0 Import CSV Files into Pandas Data Frame

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
csv_files = {
    'calendar.csv': 'calendar',
    'suppliers.csv': 'suppliers',
    'customers.csv': 'customers',
    'products.csv': 'products',
    'bom.csv': 'bom',
    'warehouses.csv': 'warehouses',
    'stores.csv': 'stores',
    'manufacturing.csv': 'manufacturing',
    'inventory_records.csv': 'inventory_records',
    'production_schedule.csv': 'production_schedule',
    'sales_orders.csv': 'sales_orders',
    'order_line_items.csv': 'order_line_items',
    'backorders.csv': 'backorders',
    'purchase_orders.csv': 'purchase_orders',
    'purchase_order_line_items.csv': 'purchase_order_line_items',
    'material_requirements.csv': 'material_requirements',
    'forecast.csv': 'forecast'
}

# Read CSV files into DataFrames
dataframes = {}
for file, table_name in csv_files.items():
    df = pd.read_csv(file)
    dataframes[table_name] = df


### 1.1 Create a SQLite Database and Import DataFrames

In [3]:
# Create a SQLite engine
engine = create_engine('sqlite:///supply_chain.db')
# Write DataFrames to SQLite database
for table_name, df in dataframes.items():
 df.to_sql(table_name, engine, if_exists='replace', index=False)

## 2.0 Load SQL Extension and Connect to Database

In [4]:
%load_ext sql
# Connect to the SQLite database
%sql sqlite:///supply_chain.db

### 2.1 View Tables

In [5]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///supply_chain.db
Done.


name
calendar
suppliers
customers
products
bom
warehouses
stores
manufacturing
inventory_records
production_schedule


## 3.0 Explore Data and Generating Insights

### 3.1. Top-Selling Products and Backorders

In [6]:
%%sql
SELECT p.product_name, SUM(oli.quantity_ordered) as total_sold
FROM order_line_items oli
JOIN products p ON oli.product_id = p.product_id
GROUP BY p.product_name
ORDER BY total_sold DESC
LIMIT 10;

 * sqlite:///supply_chain.db
Done.


product_name,total_sold
Shirts 47,48
Backpacks 282,47
Backpacks 45,45
Sleeping Bags 81,44
Sleeping Bags 86,43
Pants 32,42
Tents 177,41
Tents 14,41
Sleeping Bags 219,41
Shirts 284,41


- 'Shirts 47' is the top-selling product with 48 units sold, indicating strong customer preference.
- The top 10 products include a mix of shirts, backpacks, sleeping bags, pants, and tents, showcasing diverse high-demand items.

In [7]:
%%sql

WITH top_products AS (
    SELECT
        p.product_id,
        p.product_name,
        SUM(oli.quantity_ordered) AS total_sold
    FROM
        order_line_items oli
        JOIN products p ON oli.product_id = p.product_id
    GROUP BY
        p.product_id,
        p.product_name
    ORDER BY
        total_sold DESC
    LIMIT 10
)
SELECT
    tp.product_name,
    tp.total_sold,
    COALESCE(SUM(bo.backorder_quantity), 0) AS total_backordered
FROM
    top_products tp
    LEFT JOIN backorders bo ON tp.product_id = bo.product_id
GROUP BY
    tp.product_name,
    tp.total_sold
ORDER BY
    tp.total_sold DESC;


 * sqlite:///supply_chain.db
Done.


product_name,total_sold,total_backordered
Shirts 47,48,3515
Backpacks 282,47,3733
Backpacks 45,45,3373
Sleeping Bags 81,44,3583
Sleeping Bags 86,43,4362
Pants 32,42,4267
Shirts 187,41,4579
Sleeping Bags 219,41,4365
Tents 14,41,3573
Tents 177,41,3490


- The top-selling products like 'Shirts 47' and 'Backpacks 282' have substantial backordered quantities of 3,515 and 3,733 units respectively, highlighting a significant supply shortfall.
- The high backorder volumes across all top 10 products indicate strong demand outpacing supply, suggesting an urgent need to ramp up production and adjust inventory strategies.


### 3.1.1. Sales by City and Customer Segment

In [8]:
%%sql
SELECT
    c.city AS region,
    ROUND(SUM(so.total_order_value),2) AS total_sales
FROM
    sales_orders so
    JOIN customers c ON so.customer_id = c.customer_id
GROUP BY
    c.city
ORDER BY
    total_sales DESC;

 * sqlite:///supply_chain.db
Done.


region,total_sales
Toronto,885247.86
Vancouver,849254.82
Los Angeles,832234.55
Chicago,790753.66
New York,789462.58


- Toronto and Vancouver are the leading cities in sales, each generating over $800,000, highlighting key regional markets.


In [9]:
%%sql
SELECT
    c.customer_segment,
    ROUND(SUM(so.total_order_value),2) AS total_sales
FROM
    sales_orders so
    JOIN customers c ON so.customer_id = c.customer_id
GROUP BY
    c.customer_segment
ORDER BY
    total_sales DESC;

 * sqlite:///supply_chain.db
Done.


customer_segment,total_sales
Outdoor Enthusiast,1518502.24
Casual Shopper,1361783.29
Professional,1266667.95


- The 'Outdoor Enthusiast' segment contributes the highest sales at $1,518,502, indicating this group as a primary target market.

### 3.1.2. Inventory Levels for Reordering by Store and Warehouse

In [10]:
%%sql
SELECT
    location_type,
    location_id,
    COUNT(DISTINCT product_id) AS products_past_reorder_point
FROM
    inventory_records
WHERE
    quantity_on_hand <= reorder_point
GROUP BY
    location_type,
    location_id
ORDER BY
    location_type ASC,
    location_id ASC;

 * sqlite:///supply_chain.db
Done.


location_type,location_id,products_past_reorder_point
Store,1,135
Store,2,146
Store,3,125
Store,4,136
Store,5,148
Store,6,135
Store,7,145
Store,8,133
Store,9,134
Store,10,132


- All warehouses have 300 products past the reorder point, signaling a critical need for stock replenishment at distribution centers.
- Stores show variation in products past the reorder point, ranging from 125 to 148, suggesting differing inventory turnover rates or demand across locations.


### 3.1.3. Supplier Performance Metrics

In [11]:
%%sql
SELECT supplier_name, AVG(on_time_delivery_rate) as avg_delivery_rate
FROM suppliers
GROUP BY supplier_name
ORDER BY avg_delivery_rate DESC;

 * sqlite:///supply_chain.db
Done.


supplier_name,avg_delivery_rate
Supplier_28,0.99
Supplier_1,0.99
Supplier_15,0.98
Supplier_17,0.97
Supplier_18,0.96
Supplier_29,0.95
Supplier_13,0.94
Supplier_30,0.92
Supplier_12,0.91
Supplier_23,0.9


- Supplier_28 and Supplier_1 have the highest on-time delivery rates at 99%, indicating exceptional reliability.
- Suppliers like Supplier_10 and Supplier_6 have lower delivery rates at 70%, pointing to potential issues affecting supply chain efficiency.

### 3.1.4. Monthly Sales by Product Category

In [12]:
%%sql

SELECT
    p.category AS product_category,
    strftime('%m', so.order_date) AS month,
    ROUND(SUM(oli.line_total), 2) AS net_revenue
FROM
    sales_orders so
    JOIN order_line_items oli ON so.order_id = oli.order_id
    JOIN products p ON oli.product_id = p.product_id
    JOIN customers c ON so.customer_id = c.customer_id
WHERE
    c.customer_type = 'Wholesale'
GROUP BY
    p.category,
    month
ORDER BY
    p.category ASC,
    month ASC,
    net_revenue DESC;

 * sqlite:///supply_chain.db
Done.


product_category,month,net_revenue
Apparel,01,44295.35
Apparel,02,36792.8
Apparel,03,38067.07
Apparel,04,64617.55
Apparel,05,35484.03
Apparel,06,46495.88
Apparel,07,23406.15
Apparel,08,61071.46
Apparel,09,25176.89
Apparel,10,48485.61


- Apparel sales peaked in April and August, exceeding `$60,000` suggesting seasonal trends possibly linked to spring and late summer.
- Equipment sales saw significant increases in April and October, with net revenues over `$59,000` and `$63,000` respectively, indicating periods of heightened demand.

### 3.1.5. Rolling Monthly Sales Growth by Product Category

In [13]:
%%sql

WITH MonthlySales AS (
    SELECT
        p.category AS product_category,
        strftime('%m', so.order_date) AS order_month,
        SUM(oli.line_total) AS monthly_total_sales
    FROM
        sales_orders so
        JOIN order_line_items oli ON so.order_id = oli.order_id
        JOIN products p ON oli.product_id = p.product_id
    GROUP BY
        p.category,
        order_month
),
MonthlyGrowth AS (
    SELECT
        product_category,
        order_month,
        monthly_total_sales,
        LAG(monthly_total_sales) OVER (
            PARTITION BY product_category
            ORDER BY CAST(order_month AS INTEGER)
        ) AS prev_month_sales
    FROM
        MonthlySales
)
SELECT
    mg.product_category,
    mg.order_month,
    mg.monthly_total_sales,
    CASE
        WHEN mg.prev_month_sales IS NOT NULL AND mg.prev_month_sales != 0 THEN
            ROUND(((mg.monthly_total_sales - mg.prev_month_sales) / mg.prev_month_sales) * 100.0, 2)
        ELSE
            0.0
    END AS monthly_sales_growth_percentage
FROM
    MonthlyGrowth mg
ORDER BY
    mg.product_category ASC,
    mg.order_month ASC;

 * sqlite:///supply_chain.db
Done.


product_category,order_month,monthly_total_sales,monthly_sales_growth_percentage
Apparel,01,95598.83,0.0
Apparel,02,58678.25,-38.62
Apparel,03,57194.58,-2.53
Apparel,04,120887.71,111.36
Apparel,05,84708.94,-29.93
Apparel,06,177464.34,109.5
Apparel,07,145525.21,-18.0
Apparel,08,211761.69,45.52
Apparel,09,129995.17,-38.61
Apparel,10,139096.17,7.0


- Apparel experienced substantial growth in April (+111.36%) and June (+109.5%), but faced sharp declines in February (-38.62%) and September (-38.61%).
- Equipment saw a remarkable sales surge in April (+240.9%), yet encountered significant drops in March (-55.47%) and December (-45.7%), highlighting volatility.


### 3.1.6. Customer Segment Insights

In [14]:
%%sql

SELECT
    c.customer_segment,
    p.category AS product_category,
    SUM(oli.line_total) AS total_sales
FROM
    sales_orders so
    JOIN customers c ON so.customer_id = c.customer_id
    JOIN order_line_items oli ON so.order_id = oli.order_id
    JOIN products p ON oli.product_id = p.product_id
GROUP BY
    c.customer_segment,
    p.category
ORDER BY
    c.customer_segment ASC,
    total_sales DESC;

 * sqlite:///supply_chain.db
Done.


customer_segment,product_category,total_sales
Casual Shopper,Equipment,435006.7
Casual Shopper,Apparel,419670.92
Outdoor Enthusiast,Apparel,564373.14
Outdoor Enthusiast,Equipment,434585.37
Professional,Apparel,417015.63
Professional,Equipment,363807.13


- The 'Outdoor Enthusiast' segment is the top spender on Apparel, contributing over `$564,000` in sales, indicating strong brand engagement.
- Casual Shoppers have a balanced purchase pattern between Equipment `$435,006` and Apparel `$419,670`, suggesting diverse interests within this group.


### 3.1.7. Average Order Value by Product Category and Payment Method for Retail Clients

In [15]:
%%sql

WITH OrderTotals AS (
    SELECT
        so.order_id,
        p.category AS product_category,
        c.payment_terms AS payment_method,
        SUM(oli.line_total) AS order_total
    FROM
        sales_orders so
        JOIN order_line_items oli ON so.order_id = oli.order_id
        JOIN products p ON oli.product_id = p.product_id
        JOIN customers c ON so.customer_id = c.customer_id
    WHERE
        c.customer_type = 'Retail'
    GROUP BY
        so.order_id,
        p.category,
        c.payment_terms
)
SELECT
    product_category,
    payment_method,
    ROUND(AVG(order_total), 2) AS average_order_value
FROM
    OrderTotals
GROUP BY
    product_category,
    payment_method
ORDER BY
    product_category ASC,
    average_order_value DESC;


 * sqlite:///supply_chain.db
Done.


product_category,payment_method,average_order_value
Apparel,Prepaid,10.02
Equipment,Prepaid,9.06


- Retail clients purchasing Apparel have a slightly higher average order value `$10.02` compared to Equipment `$9.06`, indicating marginally higher spending on clothing items.
- All retail transactions are made using the Prepaid payment method, suggesting that retail customers prefer or are required to pay upfront.


### 3.1.8. Payment Method by Client Type

In [16]:
%%sql

SELECT
    c.customer_type,
    c.payment_terms AS payment_method,
    COUNT(DISTINCT so.order_id) AS order_count,
    ROUND(SUM(so.total_order_value),2) AS total_sales
FROM
    sales_orders so
    JOIN customers c ON so.customer_id = c.customer_id
GROUP BY
    c.customer_type,
    c.payment_terms
ORDER BY
    c.customer_type ASC,
    total_sales DESC;

 * sqlite:///supply_chain.db
Done.


customer_type,payment_method,order_count,total_sales
E-commerce,Prepaid,20084,1307871.38
Retail,Prepaid,58835,1505191.05
Wholesale,Net 60,21081,1333891.05


- Retail clients have the highest order count (58,835 orders) and total sales `$1,505,191`, all using the Prepaid payment method, indicating frequent small transactions and a preference for upfront payments.
- Wholesale customers use Net 60 payment terms, contributing significant sales `$1,333,891` with fewer orders (21,081), highlighting the importance of offering extended payment terms to larger clients.

### 3.1.9. Top 10 Suppliers at Risk for Delays

In [17]:
%%sql

SELECT
    supplier_id,
    supplier_name,
    lead_time_days,
    lead_time_variability,
    on_time_delivery_rate
FROM
    suppliers
ORDER BY
    lead_time_variability DESC,
    on_time_delivery_rate ASC
LIMIT 10;


 * sqlite:///supply_chain.db
Done.


supplier_id,supplier_name,lead_time_days,lead_time_variability,on_time_delivery_rate
11,Supplier_11,13,4,0.76
7,Supplier_7,8,4,0.83
19,Supplier_19,19,4,0.83
14,Supplier_14,24,4,0.88
24,Supplier_24,22,4,0.88
16,Supplier_16,11,4,0.9
22,Supplier_22,21,4,0.9
29,Supplier_29,26,4,0.95
1,Supplier_1,23,4,0.99
4,Supplier_4,13,3,0.79


- Supplier_11 and Supplier_7 have the highest lead time variability (4 days) and lower on-time delivery rates (76% and 83%), marking them as high-risk suppliers for potential delays.
- Despite a high on-time delivery rate (99%), Supplier_1 appears in the at-risk list due to a high lead time variability (4 days) and longer lead times (23 days), indicating potential scheduling challenges.

### 3.1.10. Top 10 Projected Shortages

In [18]:
%%sql

SELECT
    f.product_id,
    SUM(f.forecast_quantity) AS forecasted_demand,
    COALESCE(SUM(ir.quantity_on_hand), 0) AS total_inventory,
    (SUM(f.forecast_quantity) - COALESCE(SUM(ir.quantity_on_hand), 0)) AS projected_shortage
FROM
    forecast f
LEFT JOIN
    inventory_records ir ON f.product_id = ir.product_id
WHERE
    f.forecast_date BETWEEN DATE('2023-11-01') AND DATE('2023-11-30')
GROUP BY
    f.product_id
HAVING
    projected_shortage > 0
ORDER BY
    projected_shortage DESC
LIMIT 10;


 * sqlite:///supply_chain.db
Done.


product_id,forecasted_demand,total_inventory,projected_shortage
132,102760,7005,95755
235,101940,6195,95745
118,102060,6555,95505
12,99980,5610,94370
253,100540,6525,94015
159,99740,6885,92855
44,98340,5625,92715
274,99080,6480,92600
7,98160,5880,92280
267,99020,7140,91880


- Product ID 132 faces the highest projected shortage with a deficit of 95,755 units, signaling a critical need for inventory replenishment.
- The significant gaps between forecasted demand and current inventory levels for these top 10 products highlight an urgent need to expedite production or adjust forecasts.

### 3.1.11 Upcoming Production Schedules at Risk due to Material Shortages

In [19]:
%%sql

SELECT
    ps.production_schedule_id,
    ps.product_id,
    ps.planned_start_date,
    mr.component_id,
    mr.required_quantity,
    COALESCE(ir.quantity_on_hand, 0) AS component_on_hand,
    (mr.required_quantity - COALESCE(ir.quantity_on_hand, 0)) AS shortage_quantity
FROM
    production_schedule ps
JOIN
    material_requirements mr ON ps.production_schedule_id = mr.production_schedule_id
LEFT JOIN
    inventory_records ir ON mr.component_id = ir.product_id
WHERE
    (mr.required_quantity - COALESCE(ir.quantity_on_hand, 0)) > 0
ORDER BY
    ps.planned_start_date ASC,
    shortage_quantity DESC
LIMIT 10;


 * sqlite:///supply_chain.db
Done.


production_schedule_id,product_id,planned_start_date,component_id,required_quantity,component_on_hand,shortage_quantity
16,216,2022-01-04,1759,8445,0,8445
16,216,2022-01-04,1758,6756,0,6756
16,216,2022-01-04,1757,3378,0,3378
30,162,2022-01-06,1565,9795,0,9795
30,162,2022-01-06,1566,5877,0,5877
30,162,2022-01-06,1567,5877,0,5877
145,106,2022-01-06,1366,3335,0,3335
145,106,2022-01-06,1369,3335,0,3335
145,106,2022-01-06,1367,2668,0,2668
145,106,2022-01-06,1368,667,0,667


- Production schedules 16, 30, and 145 are at risk due to lack of inventory for required components, with shortages up to 9,795 units for components like 1565.
- Immediate action is required to procure missing components to prevent production delays and meet planned start dates.

### 3.1.12. Total Expected Incoming Inventory from Purchase Orders

In [20]:
%%sql

SELECT
    pol.component_id AS product_id,
    SUM(pol.quantity_ordered) AS incoming_quantity
FROM
    purchase_orders po
JOIN
    purchase_order_line_items pol ON po.purchase_order_id = pol.purchase_order_id
WHERE
    po.order_status = 'Pending'
    AND po.expected_delivery_date BETWEEN DATE('2023-10-01') AND DATE('2023-10-08')
GROUP BY
    pol.component_id
ORDER BY
    incoming_quantity DESC
Limit 10;


 * sqlite:///supply_chain.db
Done.


product_id,incoming_quantity
1655,1327
1591,1305
1280,1241
1068,1014
1384,1011
1836,984
2039,973
1653,950
1183,949
1639,922


- Product IDs 1655 and 1591 have the highest expected incoming quantities of 1,327 and 1,305 units respectively, indicating priority restocking for these items.
- The top 10 products are set to receive between 922 and 1,327 units each, suggesting a significant inventory replenishment to meet anticipated demand or to alleviate current shortages.